In [ ]:
!pip install --upgrade transformers
import transformers
import librosa
import torch
import IPython.display as display
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import numpy as np
!pip install huggingsound
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 890.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
#load pre-trained model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:755: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd

# creating a data frame
df = pd.read_csv("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/overview-of-recordings.csv")
df=df.drop(['speaker_id', 'writer_id', 'audio_clipping', 'audio_clipping:confidence','background_noise_audible','background_noise_audible:confidence','overall_quality_of_the_audio','quiet_speaker','quiet_speaker:confidence','file_download','prompt'], axis=1)
df.head()


,file_name,phrase
0,1249120_43453425_58166571.wav,When I remember her I feel down
1,1249120_43719934_43347848.wav,When I carry heavy things I feel like breaking...
2,1249120_43719934_53187202.wav,there is too much pain when i move my arm
3,1249120_31349958_55816195.wav,My son had his lip pierced and it is swollen a...
4,1249120_43719934_82524191.wav,My muscles in my lower back are aching


In [ ]:
directory = '/kaggle/input/medical-speech-transcription-and-intent/medical speech transcription and intent/Medical Speech, Transcription, and Intent/recordings/train'
df_new = pd.DataFrame()
#df_new.columns=['speakerId','fileName','Phrase','writerId']
# iterate over files in
# that directory
import os
#cnt=0;
for filename in os.listdir(directory):
    #if cnt<10:
    f = os.path.join(directory, filename)
        # checking if it is a file
    if os.path.isfile(f):
            #print(filename)
            #df_new=df.loc[df['file_name'] == filename]
        df_new=df_new.append(df.loc[df['file_name'] == filename])
        #cnt+=1
    else:
        break

len(df_new)

381

In [ ]:
dict1={}
lst1=list(df_new['phrase'])
lst2=list(df_new['file_name'])
#print(lst2)
final_list=[]
for i in range(len(lst1)):
    final_list.append({"path":"/kaggle/input/medical-speech-transcription-and-intent/medical speech transcription and intent/Medical Speech, Transcription, and Intent/recordings/train/"+lst2[i],"transcription":lst1[i]})
#print(final_list)

In [ ]:
from huggingsound import SpeechRecognitionModel
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english")
audio_paths = ["/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44142156_100535941.wav", "/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/train/1249120_44142156_102691000.wav"]

transcriptions = model.transcribe(audio_paths)
print(transcriptions)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

100%|██████████| 2/2 [00:10<00:00,  5.36s/it]

[{'transcription': 'i have some terrible problems when i am breathing during the night', 'start_timestamps': [720, 780, 820, 860, 920, 960, 980, 1060, 1140, 1180, 1220, 1260, 1340, 1440, 1460, 1540, 1600, 1660, 1740, 1780, 1820, 1880, 1940, 2020, 2120, 2200, 2260, 2340, 2400, 2440, 2500, 2540, 2640, 2700, 2760, 2980, 3040, 3080, 3120, 3180, 3220, 3280, 3340, 3380, 3460, 3480, 3600, 3640, 3680, 3740, 3800, 3920, 4020, 4120, 4160, 4200, 4220, 4280, 4300, 4360, 4400, 4460, 4600, 4640, 4660, 4720], 'end_timestamps': [740, 800, 840, 880, 940, 980, 1020, 1080, 1160, 1200, 1240, 1300, 1360, 1460, 1480, 1560, 1620, 1680, 1760, 1800, 1840, 1900, 1960, 2040, 2140, 2220, 2280, 2360, 2420, 2460, 2520, 2560, 2660, 2720, 2780, 3000, 3060, 3100, 3140, 3200, 3240, 3300, 3360, 3400, 3480, 3500, 3620, 3660, 3700, 3760, 3820, 3940, 4040, 4140, 4180, 4220, 4260, 4300, 4320, 4380, 4420, 4480, 4620, 4660, 4680, 4740], 'probabilities': [0.999874472618103, 0.9997643828392029, 0.9997722506523132, 0.99995291233

In [ ]:
directory = '/kaggle/input/medical-speech-transcription-and-intent/medical speech transcription and intent/Medical Speech, Transcription, and Intent/recordings/validate'
df_new_val = pd.DataFrame()
#df_new.columns=['speakerId','fileName','Phrase','writerId']
# iterate over files in
# that directory
import os
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #print(filename)
        #df_new=df.loc[df['file_name'] == filename]
        df_new_val=df_new_val.append(df.loc[df['file_name'] == filename])
len(df_new_val)


385

In [ ]:
lst1_val=list(df_new_val['phrase'])
lst2_val=list(df_new_val['file_name'])
#print(lst2)
final_list_val=[]
for i in range(len(lst1)):
    final_list_val.append({"path":"/kaggle/input/medical-speech-transcription-and-intent/medical speech transcription and intent/Medical Speech, Transcription, and Intent/recordings/validate/"+lst2_val[i],"transcription":lst1_val[i]})
#print(final_list_val)

In [ ]:
evaluation = model.evaluate(final_list_val)

print(evaluation)

100%|██████████| 381/381 [13:37<00:00,  2.15s/it]

{'cer': 0.20194265107892748, 'wer': 0.44878744650499286}


In [ ]:
os.makedirs('../op3')

In [ ]:
from huggingsound import TrainingArguments, ModelArguments, SpeechRecognitionModel, TokenSet

output_dir = "../op3"

# first of all, you need to define your model's token set
# however, the token set is only needed for non-finetuned models
# if you pass a new token set for an already finetuned model, it'll be ignored during training
tokens = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
token_set = TokenSet(tokens)

model.finetune(
    output_dir,
    train_data=final_list,
    #eval_data=final_list_val, # the eval_data is optional
    token_set=token_set,
)

  0%|          | 0/381 [00:00<?, ?ex/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1638: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,
Feature extractor saved in ../op3/preprocessor_config.json
tokenizer config file saved in ../op3/tokenizer_config.json
Special tokens file saved in ../op3/special_tokens_map.json
The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: length. If length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 381
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 144
  Number of trainable parameters = 311262369
/opt/conda/lib

Step,Training Loss
100,176.313700




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ../op3
Configuration saved in ../op3/config.json
Model weights saved in ../op3/pytorch_model.bin
loading configuration file ../op3/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "../op3",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": true,
  "diversity_loss_weight": 0.1,
  "do_stable_laye

***** train metrics *****
  epoch                    =         3.0
  total_flos               = 172609897GF
  train_loss               =     156.096
  train_runtime            =  2:28:20.03
  train_samples            =         381
  train_samples_per_second =       0.128
  train_steps_per_second   =       0.016


All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at ../op3.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Wav2Vec2ForCTC for predictions without further training.
loading configuration file ../op3/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2Processor",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

loading file vocab.json
loading file tokenizer_config.json
loading file added_tokens.json
loading file special_tokens_map.json


In [ ]:
evaluation = model.evaluate(final_list_val)

print(evaluation)

100%|██████████| 381/381 [13:36<00:00,  2.14s/it]

{'cer': 0.17105798871633804, 'wer': 0.36148359486447934}


In [ ]:
model_file = "model"
with open(model_file,'wb') as f:
    pickle.dump(model, f)

NameError: name 'model' is not defined

In [ ]:
display.Audio("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate/1249120_44246595_107248522.wav")

In [ ]:
loaded_model = pickle.load(open("/kaggle/input/speech/model.unknown", 'rb'))
print(loaded_model)
audio_paths = ["/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/validate/1249120_44246595_107248522.wav"]
transcriptions = loaded_model.transcribe(audio_paths)
print(transcriptions)

100%|██████████| 1/1 [00:02<00:00,  2.23s/it]

[{'transcription': 'notice an important decrease in my vision when try to look at things up clos', 'start_timestamps': [580, 640, 700, 760, 800, 840, 900, 940, 960, 1000, 1040, 1060, 1160, 1240, 1280, 1340, 1420, 1440, 1480, 1520, 1560, 1600, 1660, 1740, 1760, 1820, 1860, 1900, 1940, 1980, 2000, 2020, 2080, 2120, 2160, 2200, 2300, 2360, 2400, 2440, 2480, 2520, 2540, 2560, 2600, 2640, 2680, 2800, 2840, 2900, 2980, 3020, 3060, 3080, 3120, 3140, 3180, 3220, 3260, 3300, 3320, 3360, 3400, 3440, 3500, 3540, 3580, 3660, 3700, 3760, 3800, 3860, 3900, 3980, 4100, 4540], 'end_timestamps': [600, 660, 720, 780, 820, 860, 920, 960, 980, 1020, 1060, 1080, 1180, 1260, 1300, 1360, 1440, 1460, 1500, 1540, 1580, 1620, 1680, 1760, 1780, 1840, 1880, 1920, 1960, 2000, 2020, 2040, 2100, 2140, 2180, 2220, 2320, 2380, 2420, 2460, 2500, 2540, 2560, 2580, 2620, 2660, 2700, 2820, 2860, 2920, 3000, 3040, 3080, 3100, 3140, 3160, 3200, 3240, 3280, 3320, 3340, 3380, 3420, 3460, 3520, 3560, 3600, 3680, 3720, 3780, 38

In [ ]:
evaluation = loaded_model.evaluate(final_list_val)

print(evaluation)

100%|██████████| 381/381 [13:36<00:00,  2.14s/it]

{'cer': 0.17105798871633804, 'wer': 0.36148359486447934}


In [ ]:
#load audio file
#audio, sampling_rate = librosa.load("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/test/1249120_13842059_18087389.wav",sr=16000)

In [ ]:
# audio,sampling_rate
# display.Audio("/kaggle/input/medical-speech-transcription-and-intent/Medical Speech, Transcription, and Intent/recordings/test/1249120_13842059_18087389.wav", autoplay=True, rate = 22050)

In [ ]:
# input_values = tokenizer(audio, return_tensors = 'pt').input_values
# input_values

In [ ]:
# store logits (non-normalized predictions)
# logits = model(input_values).logits
# logits

In [ ]:
# store predicted id's
# pass the logit values to softmax to get the predicted values
# predicted_ids = torch.argmax(logits, dim =-1)

In [ ]:
# pass the prediction to the tokenzer decode to get the transcription
# transcriptions = tokenizer.decode(predicted_ids[0])